In [1]:
from DNABERT.motif import motif_utils
from Bio import SeqIO
import pandas as pd
import numpy as np
from Bio.Seq import Seq

In [2]:
sequence_file = "data/opJS.amplicon.long.fa"

reverse_complement = lambda x: str(Seq(x.seq).reverse_complement())


with open(sequence_file) as fasta_file:
    identifiers = []
    seqs = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):
        identifiers.append(seq_record.id)
        seq_ = reverse_complement(seq_record) # ! we reverse complement for this experiment
        seqs.append(str(seq_))

#converting lists to pandas Series    
s1 = pd.Series(identifiers, name='fragment')
s2 = pd.Series(seqs, name='seq')
#Gathering Series into a pandas DataFrame and rename index as ID column
seq_df = pd.DataFrame(dict(ID=s1, seq=s2)).set_index(['ID'])

print(f"seq_df shape {seq_df.shape}")
seq_df.head()

seq_df shape (15, 1)


,seq
ID,
opJS2_1xTetO,ATGTGTGAGTTTTGTCGGATCCGAGACGACCGCCTGATCCGCCCGT...
opJS2_2xTetO,ATGTGTGAGTTTTGTCGGATCCGAGACGACCGCCTGATCCGCCCGT...
opJS2_3xTetO,ATGTGTGAGTTTTGTCGGATCCGAGACGACCGCCTGATCCGCCCGT...
opJS2_4xTetO,ATGTGTGAGTTTTGTCGGATCCGAGACGACCGCCTGATCCGCCCGT...
opJS2_5xTetO,ATGTGTGAGTTTTGTCGGATCCGAGACGACCGCCTGATCCGCCCGT...


In [3]:
seq_df.index

Index(['opJS2_1xTetO', 'opJS2_2xTetO', 'opJS2_3xTetO', 'opJS2_4xTetO',
       'opJS2_5xTetO', 'opJS2_6xTetO', 'opJS2_7xTetO', 'opJS2_7xTetO-compact',
       'opJS2_6xTetO-compact', 'opJS2_5xTetO-compact', 'opJS2_6xTetO-spread',
       'opJS2_5xTetO-spread', 'opJS2_4xTetO-spread', 'opJS2_3xTetO-spread',
       'opJS2_2xTetO-spread'],
      dtype='object', name='ID')

In [4]:
KMER = 3

def process_sequence(seq):
    seq = str(Seq(seq).reverse_complement()) # reverse-complement the sequence
    return motif_utils.seq2kmer(seq,KMER) # split sequence into k-mers for bert

#conditions
name_c = lambda group,dox,mtase,constr: f"{group}_opJS2-{dox}{mtase}_LONG.opJS2_{constr}.full_unclustered.matrix"
group_c = ["L057","L058","L059","L060"]
dox_c = ["24hr-1000ngDox","no-dox"]
mtase_c = ["","-plusCpG",]
construct_c = [f"{i}xTetO" for i in range (1,8)]

def process_construct(group,dox,mtase,construct):
    experiment_file = name_c(group,dox,mtase,construct)
    print(experiment_file)
    file_path = f"data/{experiment_file}"
    data = pd.read_csv(file_path,sep='\t', header=[0])
    data = data.iloc[: , 1:] # remove index column
    return np.array(data.mode(0).iloc[0]) # get consensus status across reads, return it as array

In [5]:
def process_condition(dox,mtase):
    data = []
    for construct in construct_c:
        for group in group_c:
            try:
                mask = process_construct(group,dox,mtase,construct)
                sequence = seq_df.loc[f"opJS2_{construct}"].seq.lower()
                data.append([group,construct,sequence,mask])
            except:
                continue
    return pd.DataFrame(data, columns=['group','construct', 'sequence','label'])


In [6]:
# different conditions should have different dataframes
df_naming = {"24hr-1000ngDox":'dox','no-dox':'nodox',
            "":"nocpg","-plusCpG":"cpg"}
for mtase in mtase_c:
    for dox in dox_c:
        name = f'{df_naming[mtase]}_{df_naming[dox]}'
        globals()[name] = process_condition(dox,mtase)

L057_opJS2-24hr-1000ngDox_LONG.opJS2_1xTetO.full_unclustered.matrix
L058_opJS2-24hr-1000ngDox_LONG.opJS2_1xTetO.full_unclustered.matrix
L059_opJS2-24hr-1000ngDox_LONG.opJS2_1xTetO.full_unclustered.matrix
L060_opJS2-24hr-1000ngDox_LONG.opJS2_1xTetO.full_unclustered.matrix
L057_opJS2-24hr-1000ngDox_LONG.opJS2_2xTetO.full_unclustered.matrix
L058_opJS2-24hr-1000ngDox_LONG.opJS2_2xTetO.full_unclustered.matrix
L059_opJS2-24hr-1000ngDox_LONG.opJS2_2xTetO.full_unclustered.matrix
L060_opJS2-24hr-1000ngDox_LONG.opJS2_2xTetO.full_unclustered.matrix
L057_opJS2-24hr-1000ngDox_LONG.opJS2_3xTetO.full_unclustered.matrix
L058_opJS2-24hr-1000ngDox_LONG.opJS2_3xTetO.full_unclustered.matrix
L059_opJS2-24hr-1000ngDox_LONG.opJS2_3xTetO.full_unclustered.matrix
L060_opJS2-24hr-1000ngDox_LONG.opJS2_3xTetO.full_unclustered.matrix
L057_opJS2-24hr-1000ngDox_LONG.opJS2_4xTetO.full_unclustered.matrix
L058_opJS2-24hr-1000ngDox_LONG.opJS2_4xTetO.full_unclustered.matrix
L059_opJS2-24hr-1000ngDox_LONG.opJS2_4xTetO.full

In [7]:
nocpg_dox

,group,construct,sequence,label
0,L059,1xTetO,atgtgtgagttttgtcggatccgagacgaccgcctgatccgcccgt...,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,L059,2xTetO,atgtgtgagttttgtcggatccgagacgaccgcctgatccgcccgt...,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,L059,3xTetO,atgtgtgagttttgtcggatccgagacgaccgcctgatccgcccgt...,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,L059,4xTetO,atgtgtgagttttgtcggatccgagacgaccgcctgatccgcccgt...,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,L059,5xTetO,atgtgtgagttttgtcggatccgagacgaccgcctgatccgcccgt...,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
5,L059,6xTetO,atgtgtgagttttgtcggatccgagacgaccgcctgatccgcccgt...,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
6,L059,7xTetO,atgtgtgagttttgtcggatccgagacgaccgcctgatccgcccgt...,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


## Process for BERT

In [8]:
import random

def arr2kmer(seq, k):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space
    """
    kmer = [list(seq[x:x+k]) for x in range(len(seq)+1-k)]
    return kmer

context_dict = {}
c0 = 2
for c1 in ['cpg','nocpg']:
    for c2 in ['dox','nodox']:
        context_dict[f'{c1}_{c2}'] = c0
        c0 +=1
        
def add_context_vector(label,condition,how='np'):
    # 2 ways to add context vector - new position (np) and concat
    context = context_dict[condition]
    if how == 'np':
        label_ = []
        for kmer in label:
            kmer.append(context)
            label_.append(kmer)
        return label_
    else: # concat
        context = [context_dict[condition] for _ in range(KMER)]
        label_ = []
        for kmer in label:
            label_.append([kmer,context])
        return label_
        
def prep_for_dnabert(df,condition):
    for i,row in df.iterrows():
        df.at[i,'sequence'] = process_sequence(row.sequence)
        #df.at[i,'label'] = str(random.choice([0, 1])) #wrong,just trying binary labels
        df.at[i,'label'] = arr2kmer(row.label, KMER)
        df.at[i,'label'] = add_context_vector(row.label,condition)
    return df[['sequence','label']]

In [9]:
nocpg_dox_bert = prep_for_dnabert(nocpg_dox,'nocpg_dox')
nocpg_nodox_bert = prep_for_dnabert(nocpg_nodox,'nocpg_nodox')
cpg_dox_bert = prep_for_dnabert(cpg_dox,'cpg_dox')
cpg_nodox_bert = prep_for_dnabert(cpg_nodox,'cpg_nodox')

In [10]:
nocpg_nodox_bert.to_csv(f'DNABERT/sample_data/{KMER}/train.tsv',index=False,sep="\t")
nocpg_nodox_bert.to_csv(f'DNABERT/sample_data/{KMER}/dev.tsv',index=False,sep="\t") # need for evaluation, TODO - in future switch to new data

## Save files

In [12]:
import os
folder_processed = 'processed_data'
#os.mkdir(folder_processed)
#os.mkdir(f'{folder_processed}/{KMER}')

for c1 in ['cpg','nocpg']:
    for c2 in ['dox','nodox']:
        name1 = f'{c1}_{c2}'
        globals()[name1].to_csv(f'{folder_processed}/{name1}.csv',index=False)
        name2 = f'{name1}_bert'
        globals()[name2].to_csv(f'{folder_processed}/{KMER}/{name2}.tsv',sep="\t",)

In [ ]:
# cd examples

# export KMER=6
# export MODEL_PATH=6-new-12w-0
# export DATA_PATH=sample_data/ft/$KMER
# export OUTPUT_PATH=./ft/$KMER

# python run_finetune.py \
#     --model_type dna \
#     --tokenizer_name=dna$KMER \
#     --model_name_or_path $MODEL_PATH \
#     --task_name dnaprom \
#     --do_train \
#     --do_eval \
#     --data_dir $DATA_PATH \
#     --max_seq_length 100 \
#     --per_gpu_eval_batch_size=32   \
#     --per_gpu_train_batch_size=32   \
#     --learning_rate 2e-4 \
#     --num_train_epochs 5.0 \
#     --output_dir $OUTPUT_PATH \
#     --evaluate_during_training \
#     --logging_steps 100 \
#     --save_steps 4000 \
#     --warmup_percent 0.1 \
#     --hidden_dropout_prob 0.1 \
#     --overwrite_output \
#     --weight_decay 0.01 \
#     --n_process 8